Question: At what temperature does cooling of the neutral medium starts being dominated by collisionally excited Hydrogen line emission?

Most of the cooling in the neutral medium comes from CELs, in particular [CII]. Assuming that the ionisation fraction is low, and therefore that the [CII] is primarily excited by collisions with H atoms, the cooling function is: 
$\Gamma_{[CII]} = 0.06 \left( \frac{T}{100} \right)^{0.13} e^{-92/T}$, 

where $\Gamma_{[CII]}$ is in units of $10^{-25}$ erg cm$^3$ s$^{-1}$)

Cooling from Ly$\alpha$ is given by:
$\Gamma_{Ly\alpha} = 7000 \left( \frac{x_e}{10^{-3}} \right) \left( \frac{T}{8000} \right)^{-0.5} e^{-118,000/T}$


In [1]:
import numpy as np